#Lab 3: Pretrained Networks

---
In this lab, we will explore two popular pretrained models: a model that can label an image according to its content, and another that can fabricate a new image from a real image (image-to-image translation).

## Part 1: A pretrained network that recognizes the subject of an image<br>
We’ll run a state-of-the-art deep neural network that was pretrained on an object-recognition task.<br> 
<br>
The pretrained network we’ll explore here was trained on a subset of the ImageNet dataset (http://imagenet.stanford.edu).<br>
ImageNet is a very large dataset of over 14 million images maintained by Stanford University. All of the images are labeled with a hierarchy of nouns that come from the WordNet dataset (http://wordnet.princeton.edu).<br>
<br>

In [ ]:
# Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Import libraries
from torchvision import models
import torch

In [ ]:
# Directory function to examine the models
dir(models)

In [ ]:
# Instantiate a 101-layer convolutional neural network. (it downloads the weights of resnet101 trained on the ImageNet dataset, with 1.2 million images and 1,000 categories)


In [ ]:
# Architecture of resnet


In [ ]:
# Loading an image


In [ ]:
# Preprocess the images: scale the input image to 256 × 256, crop the image to 224 × 224 around the center, transform it to a tensor, and normalize its RGB (red, green, blue) components to match what was presented to the network during training


In [ ]:
# Pass the image through our preprocessing pipeline


In [ ]:
# Predict image class


In [ ]:
# Find out the label of the class that received the highest score: load a text file listing the labels in the same order they were presented to the network during training


In [ ]:
# Using the max function in PyTorch, which outputs the maximum value in a tensor as well as the indices where that maximum value occurred


In [ ]:
# Sorts the values


In [ ]:
# Load another image


In [ ]:
# Process the image and predict the class


## Part 2: A pretrained model that fakes it until it makes it<br>
The CycleGAN network has been trained on a dataset of (unrelated) horse images and zebra images extracted from the ImageNet dataset.<br> 

In [ ]:
import torch
import torch.nn as nn

class ResNetBlock(nn.Module):

    def __init__(self, dim):
        super(ResNetBlock, self).__init__()
        self.conv_block = self.build_conv_block(dim)

    def build_conv_block(self, dim):
        conv_block = []

        conv_block += [nn.ReflectionPad2d(1)]

        conv_block += [nn.Conv2d(dim, dim, kernel_size=3, padding=0, bias=True),
                       nn.InstanceNorm2d(dim),
                       nn.ReLU(True)]

        conv_block += [nn.ReflectionPad2d(1)]

        conv_block += [nn.Conv2d(dim, dim, kernel_size=3, padding=0, bias=True),
                       nn.InstanceNorm2d(dim)]

        return nn.Sequential(*conv_block)

    def forward(self, x):
        out = x + self.conv_block(x) 
        return out


class ResNetGenerator(nn.Module):

    def __init__(self, input_nc=3, output_nc=3, ngf=64, n_blocks=9): 

        assert(n_blocks >= 0)
        super(ResNetGenerator, self).__init__()

        self.input_nc = input_nc
        self.output_nc = output_nc
        self.ngf = ngf

        model = [nn.ReflectionPad2d(3),
                 nn.Conv2d(input_nc, ngf, kernel_size=7, padding=0, bias=True),
                 nn.InstanceNorm2d(ngf),
                 nn.ReLU(True)]

        n_downsampling = 2
        for i in range(n_downsampling):
            mult = 2**i
            model += [nn.Conv2d(ngf * mult, ngf * mult * 2, kernel_size=3,
                                stride=2, padding=1, bias=True),
                      nn.InstanceNorm2d(ngf * mult * 2),
                      nn.ReLU(True)]

        mult = 2**n_downsampling
        for i in range(n_blocks):
            model += [ResNetBlock(ngf * mult)]

        for i in range(n_downsampling):
            mult = 2**(n_downsampling - i)
            model += [nn.ConvTranspose2d(ngf * mult, int(ngf * mult / 2),
                                         kernel_size=3, stride=2,
                                         padding=1, output_padding=1,
                                         bias=True),
                      nn.InstanceNorm2d(int(ngf * mult / 2)),
                      nn.ReLU(True)]

        model += [nn.ReflectionPad2d(3)]
        model += [nn.Conv2d(ngf, output_nc, kernel_size=7, padding=0)]
        model += [nn.Tanh()]

        self.model = nn.Sequential(*model)

    def forward(self, input): 
        return self.model(input)

In [ ]:
# Create a generator
netG = ResNetGenerator()

In [ ]:
# Load a generator model that had been pretrained on the horse2zebra dataset, whose training set contains two sets of 1068 and 1335 images of horses and zebras, respectively
model_path = '/content/drive/MyDrive/DL_data/horse2zebra_0.4.0.pth'
model_data = torch.load(model_path)
netG.load_state_dict(model_data)

In [ ]:
# Put the network in evaluation mode


In [ ]:
# Define a few input transformations to make sure data enters the network with the right shape and size


In [ ]:
# Read the horse image


In [ ]:
# Preprocessing the image


In [ ]:
# Convert the image


In [ ]:
# Generate a html file
!jupyter nbconvert --to html "/content/drive/MyDrive/DL_lab/Lab 3: Pretrained Networks.ipynb"